In [1]:
from mpi4py import MPI
import gmsh
import numpy as np
from scipy.spatial import cKDTree
import pyvista as pv
from dolfinx import plot

# --- before the refinement loop (rank 0 only collects) ---
h_list, L2_list, L2_fem_list, rL2_list, rL2s_fem_list, H1s_list, rH1s_list = [], [], [], [], [], [], []
# L2_fem_list_pf, rL2s_fem_list_pf, H1s_list_pf, rH1s_list_pf = [], [], [], []
# L2_fem_list_lmbd, rL2s_fem_list_lmbd, H1s_list_lmbd, rH1s_list_lmbd = [], [], [], []

# plotter = pv.Plotter(shape=(2, 4), window_size=(2000, 1000))
N_ref = 10

Lx, Ly = 1.0, 1.0
# y_start, y_end = 0.0, 1.0
# x_start, x_end = 0.25, 0.75


In [3]:
from dolfinx import mesh, fem
import ufl
import numpy as np
from dolfinx.io import gmsh as gmshio
from mpi4py import MPI

# ---------------------------------------------------------------------
# 1. Mesh and tags (same mesh as reference)
# ---------------------------------------------------------------------
ref = 2
filename = f"regular_mesh_{ref}.msh"
msh, cell_markers, facet_markers = gmshio.read_from_msh(filename, MPI.COMM_WORLD, 0, gdim=2)[0:3]

tdim = msh.topology.dim  # 2
fdim = tdim - 1          # 1

lc = 1.0 / (2**ref)      # mesh size
h = lc

omega = msh

# ---------------------------------------------------------------------
# 2. Function space (standard CG on the bulk only)
# ---------------------------------------------------------------------
order = 2
V = fem.functionspace(omega, ("Lagrange", order))

# ---------------------------------------------------------------------
# 3. Trial / Test / Solution function
# ---------------------------------------------------------------------
phi = ufl.TestFunction(V)
p = fem.Function(V, name="p")
dp = ufl.TrialFunction(V)

# ---------------------------------------------------------------------
# 4. Coordinates and data
# ---------------------------------------------------------------------
x = ufl.SpatialCoordinate(omega)
# f_m = fem.Constant(omega, 0.0)
# f_m = 2*(x[1] - x[1]**2 + x[0] - x[0]**2)
f_m = x[1]
# create function
k_m = fem.Function(V, name="kappa")

def k_callable(x):
    # x is array with shape (gdim, N). compute scalar per point, return shape (1,N)
    vals = 2*(x[0])
    return vals[np.newaxis, :]

k_m.interpolate(k_callable)

kappa = 1.0
# k_m = fem.Constant(omega, kappa)  # bulk permeability

dx = ufl.Measure("dx", domain=omega)

# compute domain bbox to locate sides
coords = omega.geometry.x
xx = coords[:, 0]
yy = coords[:, 1]
xmin, xmax = xx.min(), xx.max()
ymin, ymax = yy.min(), yy.max()

tol = 1e-10 * max(xmax - xmin, ymax - ymin)

# locate boundary facets for bottom (y = ymin) and top (y = ymax)
bottom_facets = mesh.locate_entities_boundary(
    omega, fdim, lambda x: np.isclose(x[1], ymin, atol=tol)
)
top_facets = mesh.locate_entities_boundary(
    omega, fdim, lambda x: np.isclose(x[1], ymax, atol=tol)
)

# create meshtags for those Neumann parts (optional — kept for clarity)
indices = np.concatenate([bottom_facets, top_facets]).astype(np.int32)
values = np.concatenate([
    np.full(bottom_facets.shape, 10, dtype=np.int32),  # bottom tag = 10
    np.full(top_facets.shape,    20, dtype=np.int32),  # top tag    = 20
])
neumann_tags = mesh.meshtags(omega, fdim, indices, values)

# boundary measure (we'll use ds over the whole boundary with conditional g_N)
dsN = ufl.Measure("ds", domain=omega)  # we integrate g_N over the boundary

# Neumann data as conditional on the y coordinate (same as reference)
# g_N = ufl.conditional(
#     ufl.lt(abs(x[1] - ymax), tol),
#     -1.0,                                # top
#     ufl.conditional(
#         ufl.lt(abs(x[1] - ymin), tol),
#         2.0,                             # bottom
#         0.0                              # elsewhere
#     ),
# )

# ---------------------------------------------------------------------
# 5. Weak form (standard Darcy / Poisson form)
# ---------------------------------------------------------------------
a = ufl.inner(k_m * ufl.grad(dp), ufl.grad(phi)) * dx
L = f_m * phi * dx
# L -= g_N * phi * dsN

# Form for assembling (bilinear/trilinear functions expect TrialFunction in a)
A_form = ufl.inner(k_m * ufl.grad(dp), ufl.grad(phi)) * dx
b_form = L

# ---------------------------------------------------------------------
# 6. Dirichlet BCs (left/right like reference)
# ---------------------------------------------------------------------
# locate dofs on each side
left_dofs   = fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[0], xmin, atol=tol))
right_dofs  = fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[0], xmax, atol=tol))
top_dofs  = fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[1], ymax, atol=tol))
bottom_dofs  = fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[1], ymin, atol=tol))

all_dofs = np.unique(np.concatenate([left_dofs, right_dofs]))
dof_coords = V.tabulate_dof_coordinates()

p_bc_fun = fem.Function(V)
p_bc_fun.x.array[left_dofs] = 0.0
p_bc_fun.x.array[right_dofs] = dof_coords[right_dofs, 1] / 2.0
# p_bc_fun.x.array[top_dofs] = 0.0
# p_bc_fun.x.array[bottom_dofs] = 0.0

bc = fem.dirichletbc(p_bc_fun, all_dofs)
bcs = [bc]

# ---------------------------------------------------------------------
# 7. Solve linear problem using dolfinx.fem.petsc.LinearProblem
# ---------------------------------------------------------------------
from dolfinx.fem import petsc

linear_problem = petsc.LinearProblem(
    a=A_form,
    L=b_form,
    bcs=bcs,
    petsc_options={
        "ksp_type": "preonly",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
        "ksp_error_if_not_converged": True,
    },
    petsc_options_prefix="pmix_"
)

p_sol = linear_problem.solve()
p_sol.name = "pressure"

# optionally copy into p Function if you prefer
p.x.array[:] = p_sol.x.array[:]

# ---------------------------------------------------------------------
# 8. Diagnostics
# ---------------------------------------------------------------------
print("DOFs in V:", V.dofmap.index_map.size_global)


Info    : Reading 'regular_mesh_2.msh'...
Info    : 153 entities
Info    : 25 nodes
Info    : 153 elements
Info    : Done reading 'regular_mesh_2.msh'
DOFs in V: 81


In [4]:
from dolfinx import geometry
def eval_scalar_at_point(u, point):
    """
    Evaluate a scalar dolfinx.fem.Function `u` at a single physical point.

    Parameters
    ----------
    u : dolfinx.fem.Function or float/int
    point : sequence of length mesh.dim

    Returns
    -------
    float
    """
    # allow plain scalars
    if isinstance(u, (float, int, np.floating, np.integer, np.float64)):
        return float(u)
    point = np.asarray(point)
    if point.ndim == 1:
        point = point.reshape(-1, 1)
    V = u.function_space
    mesh = V.mesh
    tdim = mesh.topology.dim
    gdim = mesh.geometry.dim

    # X = np.asarray(point, dtype=np.float64).reshape(1, -1)  # shape (1, dim)
    _gdim, N = point.shape
    if _gdim != gdim:
        raise ValueError(f"Point array has gdim={_gdim}, expected {gdim}")

    # dolfinx expects 3D coordinates; pad in 2D
    if gdim == 2:
        X = np.column_stack([point.T, np.zeros(N, dtype=np.float64)])  # (N, 3)
    elif gdim == 3:
        X = point.T.copy()                                             # (N, 3)
    else:
        raise ValueError(f"Unsupported geometric dimension gdim={gdim}")

    # 1) bounding-box tree + collision test
    bbt = geometry.bb_tree(mesh, tdim)
    cands = geometry.compute_collisions_points(bbt, X)
    hits = geometry.compute_colliding_cells(mesh, cands, X)

    # pick first hit if any
    cell = -1
    links = hits.links(0)
    if len(links) > 0:
        cell = int(links[0])

    # 2) fallback: nearest entity (robust for boundary points / tiny roundoff)
    if cell == -1:
        mid_tree = geometry.create_midpoint_tree(
            mesh, tdim,
            np.arange(mesh.topology.index_map(tdim).size_local, dtype=np.int32)
        )
        nearest = geometry.compute_closest_entity(bbt, mid_tree, mesh, X)
        cell = int(nearest[0])

    # 3) Evaluate using dolfinx Function.eval
    # Try the common signature that returns an array: val = u.eval(X, cells)
    try:
        vals = np.asarray(u.eval(X, np.array([cell], dtype=np.int32))).ravel()
        return float(vals[0])
    except Exception:
        # Alternative signature: u.eval(values_buffer, X, cell)
        # Prepare buffer of correct size (value_size per point)
        vsize = getattr(u, "value_size", 1)
        buf = np.zeros((vsize, 1), dtype=np.float64) if vsize > 1 else np.zeros(1, dtype=np.float64)
        try:
            # note: some dolfinx versions expect (buf, X, cell)
            u.eval(buf, X, cell)
            return float(np.asarray(buf).ravel()[0])
        except Exception as e:
            raise RuntimeError(f"Failed to evaluate Function at point {point}: {e}")

In [5]:
# Access mesh geometry
tdim = omega.topology.dim
# assert tdim == 2, "This implementation is for triangular 2D meshes"
# assert V.ufl_element().degree == 1, "Only P1 supported in this implementation"

# Gather coordinates and connectivity
omega_geometry = omega.geometry.x[:,0:2]  # global array of vertex coords
# Build cell->vertex mapping (fast access)
# Using dolfinx.Cell to get connectivity is more robust, but below uses mesh.topology
cells = omega.topology.connectivity(0, tdim)  # vertices -> cells
# Instead get cell->vertex mapping via mesh.topology.index_map?
# Simpler: use mesh.geometry.x with mesh.topology.connectivity(tdim, 0)
ctv = omega.topology.connectivity(tdim, 0)
if ctv is None:
    omega.topology.create_connectivity(tdim, 0)
    ctv = omega.topology.connectivity(tdim, 0)
# ctv.array() gives flattened vertex indices -> need per-cell view
# There's a helper to get cell entity indices
num_cells = omega.topology.index_map(tdim).size_local  # per-rank local count
# We'll iterate over local cells only
local_cells = np.arange(num_cells, dtype=np.int32)

# For evaluating u_h and kappa, we can use u_h.x.array if needed.
# But easier: create a point evaluation lambda to get value at a point
from dolfinx.fem import Function
u_func = p_sol

# Precompute reference basis grads in physical coords per cell:
# For P1, grad phi_j is constant on cell; we can compute using inverse Jacobian
# For each local cell:
local_fluxes = {}
V_dofmap = V.dofmap.index_map.local_range  # not directly used; we'll query dofs per cell

# Get dof indices per cell
# Using dolfinx.fem.locate_dofs_topological is one option, but below we get cell_dofs
dofmap = V.dofmap
# iterate over local cells using mesh.topology.connectivity(tdim, 0)
# cell_to_vertices = ctv.links().reshape((-1, 3))  # might be shape (num_cells, 3)
# cell_to_vertices = np.array([ctv.links(c) for c in range(ctv.num_nodes)])

# But to avoid uncertain reshape semantics, use mesh.topology.connectivity(tdim, 0).entities
# Simpler robust approach: use dolfinx.mesh.cells() to get local cells vertex indices
try:
    from dolfinx.mesh import cells as dolfinx_cells
    local_cell_vertices = dolfinx_cells(mesh)
except Exception:
    # fallback: create by slicing connectivity
    # arr = ctv.array
    # local_cell_vertices = arr.reshape((-1, 3))
    local_cell_vertices = np.array([ctv.links(c) for c in range(ctv.num_nodes)])

In [6]:
ref_grads = np.array([[-1.0, -1.0],[1.0, 0.0],[0.0, 1.0]])
elem_grad_u = {}
elem_area = {}
elem_kappa_cent = {}
from tqdm import tqdm

In [ ]:
import basix
coef_glob = {i: [] for i in range(len(omega.geometry.x))}
# Iterate local cells
for local_cell_idx, cell_verts in enumerate(tqdm(local_cell_vertices)):
    # cell_verts are global vertex indices
    coords = omega_geometry[cell_verts]  # shape (3,2)
    centroid = np.mean(coords, axis=0).reshape(1,-1)

    loc_cells = np.array([[0, 1, 2]], dtype=np.int64)

    gdim = coords.shape[1]
    coord_el = basix.ufl.element("Lagrange", "triangle", 1, shape=(gdim,))

    local_mesh = mesh.create_mesh(
        MPI.COMM_SELF,
        loc_cells,
        coord_el,   # <-- this is the required "e" argument
        coords
    )

    Vloc = fem.functionspace(local_mesh, ("CG", 2))

    element = Vloc.element.basix_element

    # Get physical coordinates of cell vertices
    # cell_geometry = mesh.geometry.x[
    #     mesh.topology.connectivity(mesh.topology.dim, 0).links(local_cell_idx)
    # ]

    # ---- IMPORTANT PART ----
    # Pull back physical -> reference coordinates
    # reference_points = mesh.geometry.cmap.pull_back(
    #     x_phys, cell_geometry
    # )

  0%|          | 0/32 [01:51<?, ?it/s]


AttributeError: module 'dolfinx.mesh' has no attribute 'geometry'